### The results from the PCA alalysis is then rasterized into Geotiff files, which would be later displayed in ArcGIS

In [ ]:
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage import exposure
import os
from osgeo import gdal

In [13]:
# load the csv file of multiple deprivation degrees
df = pd.read_csv(r"C:\Users\EQiLu\MSc Thesis\PCA results\0318\builtup_all_variables_score.csv")

In [14]:
print(df)

       rows  columns   FAC1_1   FAC2_1   FAC3_1   FAC4_1   FAC5_1   FAC6_1  \
0         1      284  0.34835  1.07187 -1.67636 -0.66810 -0.48305  0.03390   
1         3      271 -0.04883  0.75246 -1.32319 -0.43809 -0.60566 -0.03077   
2         3      272  0.05303  0.72162 -1.49915 -0.43077 -0.56450  0.05425   
3         3      297  0.16747  1.16711 -1.23108 -0.87211 -0.51496 -0.00111   
4         3      298  0.15836  1.14421 -1.28117 -0.69291 -0.55032 -0.00457   
...     ...      ...      ...      ...      ...      ...      ...      ...   
45721   251      125 -0.01558  0.12814 -0.62016 -0.83387  0.40006  0.23757   
45722   251      130 -0.28080 -0.12374 -0.72869  0.62742  0.34067 -0.29328   
45723   252      125 -0.02582  0.10826 -0.60194 -0.50748  0.38875  0.04901   
45724   252      129 -0.20575  0.05413 -0.40975 -0.64698  0.37265  0.06379   
45725   252      130 -0.32578 -0.00275 -0.48670  0.08561  0.31955 -0.14016   

       sumed score  
0         -0.86111  
1         -0.45496  


In [15]:
# create the base layer, a 2-d dataframe of the same width*height at 100m grid as the original geotiff

base = pd.DataFrame(index = range(1,254) , columns= range(1,435))
print(base)
print(base.shape)

     1    2    3    4    5    6    7    8    9    10   ...  425  426  427  \
1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
5    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
249  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
250  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
251  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
252  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
253  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

     428  429  430  431  432  433  434  
1    NaN  NaN  NaN  NaN  NaN  NaN 

In [16]:
# Pass along the deprivation indices accoording to the (row,column) coordinates.  

for row in df.itertuples(index = False):
    base.at[row[0]+1,row[1]+1] = row[8]

In [17]:
print(base)
print(base.shape)

     1    2    3    4    5    6    7    8    9    10   ...  425  426  427  \
1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
5    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
249  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
250  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
251  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
252  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
253  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

     428  429  430  431  432  433  434  
1    NaN  NaN  NaN  NaN  NaN  NaN 

In [18]:
# convert the pandas dataframe to numpy array

final = base.to_numpy()
print(final)
print(final.dtype)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
object


In [19]:
# chenge the datatype into numerical

final = final.astype(float)
final.dtype

dtype('float64')

In [20]:
from osgeo import osr

In [21]:
def array2raster(newRasterfn, dataset, array, dtype):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    cols = array.shape[1]
    rows = array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform() 

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")

    # set number of band.
    if array.ndim == 2:
        band_num = 1
    else:
        band_num = array.shape[2]

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))

    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[:,:,b])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [22]:
ori_data = gdal.Open(r'C:\Users\EQiLu\MSc Thesis\100m Input March\nai_OSM_dis_major_roads_2020.tif')

In [23]:
array2raster(r'C:\Users\EQiLu\MSc Thesis\Result Map\0318\builtup_all_variables_score.tif', ori_data, final,"Float32")